In [2]:
#IMPORTO LIBRERIE
import pandas as pd

In [3]:
labeled_df = pd.read_csv('labeled_df.csv')
labeled_df.shape

(90960, 16)

In [4]:
labeled_df = labeled_df.drop(columns='Unnamed: 0')
labeled_df.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,93000801,2021-09-15 16:13:35.147,6,150.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
1,93000801,2021-08-24 10:34:24.487,5,3200.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
2,93000801,2021-09-14 16:21:43.33,6,8350.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
3,93001001,2021-06-30 09:07:16.733,6,500.00,3f241dfcc38064261fec0f303c2784e5,1,0,5,17,123,1781,90,8,99,10
4,93001001,2021-07-12 16:36:02.857,6,1895.68,a5d1eaffa8e325d370665f43624339a1,1,0,5,17,285,1781,90,8,99,10


In [5]:
from pyod.models.ecod import ECOD
X=labeled_df.copy()
X=X.drop(columns=['businessName','customerId','timestamp'])
clf = ECOD(contamination=0.001)
clf.fit(X)
clf.get_params()

{'contamination': 0.001, 'n_jobs': 1}

In [6]:
import pickle
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [7]:
def inverse_and_scores(labeled_df,enc_dict,columns,model):
    df_copy=labeled_df.copy()
    labels=model.labels_
    scores = model.decision_scores_

    for clmn in columns:
        df_copy[clmn]=enc_dict[clmn].inverse_transform(df_copy[clmn])

    df_copy['scores'] = scores
    df_copy['is_anomaly'] = labels

    return df_copy

df_scores=inverse_and_scores(labeled_df,encoder_dict,columns_to_label,clf)

df_scores.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores,is_anomaly
0,93000801,2021-09-15 16:13:35.147,Italy,150.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,13.619869,0
1,93000801,2021-08-24 10:34:24.487,Greece,3200.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,17.579564,0
2,93000801,2021-09-14 16:21:43.33,Italy,8350.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,14.067937,0
3,93001001,2021-06-30 09:07:16.733,Italy,500.00,3f241dfcc38064261fec0f303c2784e5,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,8.099064,0
4,93001001,2021-07-12 16:36:02.857,Italy,1895.68,a5d1eaffa8e325d370665f43624339a1,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,7.269080,0


In [8]:
df_scores[df_scores['is_anomaly']==1].sort_values('scores',ascending=False)

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores,is_anomaly
42632,45915153,2021-08-24 11:47:38.077,Italy,299.00,73322bb678a4c1b6a94fe29411a5fafe,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,38.141232,1
42622,45915153,2021-09-02 15:44:16.347,Italy,687.00,73322bb678a4c1b6a94fe29411a5fafe,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,37.608201,1
42619,45915153,2021-07-30 15:25:38.557,Italy,1000.00,f2a85ebb8645443730a3fba554a69914,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,37.388342,1
42623,45915153,2021-04-29 14:17:14.26,Italy,1000.00,f2a85ebb8645443730a3fba554a69914,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,37.388342,1
62085,68863923,2021-08-03 14:03:09.58,Italy,8.20,6c851139ea78e812d0d4cb33b9603687,1,Chrome,Mac OS X,BE,0,B6R6Z,39053,288,0,SRL,36.408063,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18803,1886395602,2021-07-27 11:15:56.77,Italy,1200.00,c1acc152c10a3e7c3297d0bdfa246acd,1,Chrome,Mac OS X,NL,INGB,AMU3P,39100,615,11,SS,28.589932,1
51569,49901793,2021-09-14 09:28:50.683,Italy,17.66,2a3bfe83f75de70adb71d35281bf6987,0,Mobile Safari UI/WKWebView,iOS,IT,3069,BH4SA,36061,480,11,SAS,28.582268,1
75345,85917346,2021-08-23 13:13:28.537,Italy,130.00,1033adca2d07691863d4e65a1c2f68a0,0,Chrome Mobile iOS,iOS,IT,5856,BZGLV,39100,501,0,SDF,28.578828,1
55421,62276698,2021-05-18 19:27:34.503,Italy,833.84,e0af727f9ff610a040d4745e8005a458,0,Other,Linux,DE,46660022,BZY2P,39010,615,630,SDF,28.571853,1


In [9]:
df_scores.to_csv('ECOD_scores.csv')

In [9]:
ECOD_labeled_df = X.copy()
ECOD_labeled_df['label']=clf.labels_
ECOD_labeled_df.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,label
0,6,150.00,1,4,5,17,699,2013,144,19,113,5,0
1,5,3200.00,1,4,5,17,699,2013,144,19,113,5,0
2,6,8350.00,1,4,5,17,699,2013,144,19,113,5,0
3,6,500.00,1,0,5,17,123,1781,90,8,99,10,0
4,6,1895.68,1,0,5,17,285,1781,90,8,99,10,0


In [10]:
ECOD_labeled_df.to_csv('ECOD_labeled_df.csv')